In [5]:
import pandas as pd
dados = pd.read_csv('./aborto-consolidated-parent-based.tsv', sep='\t', decimal = ',', encoding = 'UTF-8')
dados.head(3)

,target_id,target_message,id,parent_id,author,parent_name,parent_message,message,parent_label,target_parent_message,target_and_message,label
0,1,A paternidade deveria ser considerada nas disc...,Alvo,Alvo,Alvo,Alvo,Alvo,A paternidade deveria ser considerada nas disc...,Alvo da conversa,A paternidade deveria ser considerada nas disc...,A paternidade deveria ser considerada nas disc...,Alvo da conversa
1,1,A paternidade deveria ser considerada nas disc...,1,NaN,Barbara_Charles,Raiz,NaN,"E a paternidade, Sra. Ministra ? Considerando ...",NaN,A paternidade deveria ser considerada nas disc...,A paternidade deveria ser considerada nas disc...,NaN
2,1,A paternidade deveria ser considerada nas disc...,2,1,Mary_Lane,Barbara_Charles,"E a paternidade, Sra. Ministra ? Considerando ...","É engraçado que, se formos seguir a lógica do ...",NaN,A paternidade deveria ser considerada nas disc...,A paternidade deveria ser considerada nas disc...,Concorda


Tratamento dos dados, tornando-os prontos para formatar em um prompt

In [6]:
dados = dados[(dados['label'] != 'Alvo da conversa') & (dados['label'] != 'Comentário Original') & (dados['label'].notna())]


dados['label'] = dados['label'].replace('Discute', 'Outros')
dados['label'] = dados['label'].replace('Irrelevante', 'Outros')
dados['label'] = dados['label'].replace('Pede Informações', 'Outros')

dados['parent_label'] = dados['parent_label'].replace('Discute', 'Outros')
dados['parent_label'] = dados['parent_label'].replace('Irrelevante', 'Outros')
dados['parent_label'] = dados['parent_label'].replace('Pede Informações', 'Outros')

In [8]:
dados.head(5)

,target_id,target_message,id,parent_id,author,parent_name,parent_message,message,parent_label,target_parent_message,target_and_message,label
2,1,A paternidade deveria ser considerada nas disc...,2,1,Mary_Lane,Barbara_Charles,"E a paternidade, Sra. Ministra ? Considerando ...","É engraçado que, se formos seguir a lógica do ...",NaN,A paternidade deveria ser considerada nas disc...,A paternidade deveria ser considerada nas disc...,Concorda
3,1,A paternidade deveria ser considerada nas disc...,3,2,Karen_Poole,Mary_Lane,"É engraçado que, se formos seguir a lógica do ...","Na hora de abortar é escolha da mulher, na hor...",Concorda,A paternidade deveria ser considerada nas disc...,A paternidade deveria ser considerada nas disc...,Concorda
4,1,A paternidade deveria ser considerada nas disc...,4,3,Barbara_Charles,Karen_Poole,"Na hora de abortar é escolha da mulher, na hor...",Sabe qual é o bizarro em um contexto onde o ab...,Concorda,A paternidade deveria ser considerada nas disc...,A paternidade deveria ser considerada nas disc...,Concorda
5,1,A paternidade deveria ser considerada nas disc...,5,4,Stephanie_Owens,Barbara_Charles,Sabe qual é o bizarro em um contexto onde o ab...,"Alguns pontos, concordo que em caso do aborto ...",Concorda,A paternidade deveria ser considerada nas disc...,A paternidade deveria ser considerada nas disc...,Concorda
6,1,A paternidade deveria ser considerada nas disc...,6,5,Barbara_Charles,Stephanie_Owens,"Alguns pontos, concordo que em caso do aborto ...",Em relação ao último ponto é para evitar que a...,Concorda,A paternidade deveria ser considerada nas disc...,A paternidade deveria ser considerada nas disc...,Outros


In [25]:
# visualizando exemplos de cada label, para montar o few shot
exemplos_labels = exemplos = dados.groupby('label').apply(lambda x: x.sample(3)).reset_index(drop=True)


C:\Users\digog\AppData\Local\Temp\ipykernel_14808\3012924072.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  exemplos_labels = exemplos = dados.groupby('label').apply(lambda x: x.sample(3)).reset_index(drop=True)


In [26]:
exemplos_labels

,target_id,target_message,id,parent_id,author,parent_name,parent_message,message,parent_label,target_parent_message,target_and_message,label
0,41,Eu acho que a discussão sobre o aborto é muito...,2,1,Mary_Watson,Beverly_Black,"Olha, a maior parte da problemática dessa disc...",Perfeito o seu comentário! Eu adicionaria ap...,Comentário Original,Eu acho que a discussão sobre o aborto é muito...,Eu acho que a discussão sobre o aborto é muito...,Concorda
1,45,Eu acho que o período de tempo limite para o a...,3,2,Albert_Savage,Danielle_Cannon,"Sim, o espermatozoide e o óvulo não são capaze...",A capacidade de sobreviver é irrelevante. O óv...,Discorda,Eu acho que o período de tempo limite para o a...,Eu acho que o período de tempo limite para o a...,Concorda
2,1,A paternidade deveria ser considerada nas disc...,9,3,Thomas_Mueller,Karen_Poole,"Na hora de abortar é escolha da mulher, na hor...","Ué. Se a mulher tem direito ao aborto, e decid...",Concorda,A paternidade deveria ser considerada nas disc...,A paternidade deveria ser considerada nas disc...,Concorda
3,6,"Em um mundo ideal, cada pessoa pode escolher s...",2,1,Joseph_Durham,Anthony_Ward,"Em ummundo ideal, eu acho que não tinha que te...","Ah sim, no mundo ideal quem não consegue fecha...",NaN,"Em um mundo ideal, cada pessoa pode escolher s...","Em um mundo ideal, cada pessoa pode escolher s...",Discorda
4,52,"Eu acho que o aborto não é aceitável, porém a ...",11,10,Beverly_Black,Ronald_Shaw,Amigo vc deveria estar falando isso no r/consp...,Na verdade eu deveria estar falando aqui porqu...,Concorda,"Eu acho que o aborto não é aceitável, porém a ...","Eu acho que o aborto não é aceitável, porém a ...",Discorda
5,33,Eu acho que a pessoa que é contra o aborto est...,3,2,David_Fuller,Danielle_Cannon,"De fato, mas acredito que fetos são vivos, o q...",Ser um ser vivo é completamente irrelevante. U...,Concorda,Eu acho que a pessoa que é contra o aborto est...,Eu acho que a pessoa que é contra o aborto est...,Discorda
6,53,Eu acho que a questão do direito a vida e a pr...,8,7,Danielle_Cannon,Matthew_Mahoney,"Um ser vivo da espécie ""Homo Sapiens""","Defina ""espécie Homo sapiens"".",Outros,Eu acho que a questão do direito a vida e a pr...,Eu acho que a questão do direito a vida e a pr...,Outros
7,32,"Eu acho que o aborto é aceitável, a decisão de...",25,24,Ronald_Shaw,Ronald_Nicholson,Eu ficaria transtornado se minha parceira quis...,Se tivesse uma tecnologia que pudesse transpla...,Discorda,"Eu acho que o aborto é aceitável, a decisão de...","Eu acho que o aborto é aceitável, a decisão de...",Outros
8,2,Hoje em dia o aborto é permitido em casos de e...,19,18,Ryan_Davidson,David_Morgan,[deleted],"Só falou bosta, amigo. Tá ok, se é o que você ...",Outros,Hoje em dia o aborto é permitido em casos de e...,Hoje em dia o aborto é permitido em casos de e...,Outros


In [27]:
exemplo_concorda = dados[(dados['id'] == '15') & (dados['parent_id'] == '13') & (dados['target_id'] == 25)]

exemplo_concorda

,target_id,target_message,id,parent_id,author,parent_name,parent_message,message,parent_label,target_parent_message,target_and_message,label
260,25,Eu acho que a criminalização do aborto é um ex...,15,13,Brittany_Hudson,Ronnie_Simmons,Então você é a favor da vida ou contra? Nã ent...,Não sou nem a favor nem contra a vida. Sou a f...,Discorda,Eu acho que a criminalização do aborto é um ex...,Eu acho que a criminalização do aborto é um ex...,Concorda


In [38]:
# 1. Extraia o texto da primeira linha da coluna 'target_message'
texto_completo = exemplo_outros['message'].iloc[0]

# 2. Use o print para exibir a string completa, sem cortes
print(texto_completo)

Defina "espécie Homo sapiens".


In [28]:
exemplo_discorda = dados[(dados['id'] == '7') & (dados['parent_id'] == '6') & (dados['target_id'] == 40)]
exemplo_discorda

,target_id,target_message,id,parent_id,author,parent_name,parent_message,message,parent_label,target_parent_message,target_and_message,label
506,40,O aborto não é aceitável pois um feto é uma vida,7,6,Danielle_Cannon,Beverly_Black,Existem critérios sólidos e bem fundamentados ...,O DNA é um critério relativamente bem fundamen...,Concorda,O aborto não é aceitável pois um feto é uma vi...,O aborto não é aceitável pois um feto é uma vi...,Discorda


In [ ]:
exemplo_outros = dados[(dados['id'] == '8') & (dados['parent_id'] == '7') & (dados['target_id'] == 53)]

,target_id,target_message,id,parent_id,author,parent_name,parent_message,message,parent_label,target_parent_message,target_and_message,label
638,53,Eu acho que a questão do direito a vida e a pr...,8,7,Danielle_Cannon,Matthew_Mahoney,"Um ser vivo da espécie ""Homo Sapiens""","Defina ""espécie Homo sapiens"".",Outros,Eu acho que a questão do direito a vida e a pr...,Eu acho que a questão do direito a vida e a pr...,Outros


# Criação do prompt System e User

##### Rascunho do prompt system
Você é um detector contextual de posicionamento, a sua função é, dada uma mensagem e o seu contexto, classificar o posicionamento em relação a mensagem de origem.
As possíveis classificações são: 
1. Concorda - se a mensagem atual concorda com o posicionamento da mensagem origem
2. Discorda - se a mensagem atual discroda com o posicionamento da mensagem origem
3. Outros - se a mensagem atual discute, é irrelevante ou pede informações no contexto da conversa

Aqui está um exemplo de cada uma das classificações:

CONCORDA:
Mensagem original: Eu acho que a criminalização do aborto é um exemplo claro de como a religião pode influenciar a política de forma injusta. A ciência é clara: o feto tem um cérebro desenvolvido e é considerado uma pessoa, mas isso não significa que a mulher não tenha direito de escolher sobre seu próprio corpo. É importante lembrar que a vida é complexa e que existem muitos fatores a considerar, incluindo a saúde e o bem-estar da mulher.

Mensagem atual: Não sou nem a favor nem contra a vida. Sou a favor de preservarmos ou não a vida de acordo com critérios objetivos. Como eu disse, apontar que um feto é uma vida não é um argumento suficiente para criminalizar o aborto, já que plantas e animais também são formas de vida e nem por isso deixamos de matá-los. Matamos animais e plantas por motivos justificáveis e de acordo com critérios objetivos. Da mesma forma, a decisão sobre criminalizar ou descriminalizar o aborto deve seguir critérios objetivos.   >Se é a favor da morte, e usa o animais e plantas como equivalentes ao ser humano, isso significa que acha a sua própria vida com o mesmo valor que de um animal ou planta, correto? Logo, qualquer pessoa poderia te tratar com um deles, certo?  Errado. Não sou a favor da morte e nem estou usando animais e plantas como equivalentes ao ser humano. Não acho que minha vida tem o mesmo valor que a vida de um animal ou planta. Nós, como seres humanos, somos capazes de estabelecer pactos civilizatórios, e é por isso que nós prezamos por não nos matarmos. Não é sobre valor, e sim sobre uma diferença de natureza que nos torna capazes de pactuarmos juridicamente.   Enfim, sua tentativa de argumentação foi bastante pretensiosa. Você me atribuiu um monte de ideias que eu em nenhum momento expressei. Completamente presunçoso.

DISCORDA:
Mensagem original: O aborto não é aceitável pois um feto é uma vida

Mensagem atual:O DNA é um critério relativamente bem fundamentado, mas não é suficientemente sólido. Em primeiro lugar, não está claro em que momento uma determinada configuração genética passa de não-humana para humana. Em segundo lugar, faz sentido afirmar que o DNA integra todas as propriedades tipicamente exigidas para considerarmos um ser como humano? Podemos dividir nossa espécie das demais com algum nível de precisão utilizando esse método, mas não parece clara a relação de necessidade lógica entre ser da espécie Homo sapiens e possuir direito à vida. Ao meu ver, outras características parecem mais relevantes para se atribuir esse direito.

OUTROS:
Mensagem original: Eu acho que a questão do direito a vida e a propriedade é fundamental para entender a questão do aborto. Se o direito a vida e a propriedade são relativizados, então isso pode levar a uma permissividade em outros crimes, como o estupro. Isso me faz questionar se o aborto é realmente aceitável, pois se considerarmos que a vida humana é sagrada, então matar um inocente, seja um feto ou uma pessoa adulta, não pode ser aceitável

Mensagem atual: Defina "espécie Homo sapiens".

Você deve devolver primeiro a classificação, e posteriormente a justificativa.

##### Prompt system final

"""
Você é um detector contextual de posicionamento. Sua função é analisar uma mensagem atual, levando em consideração:

1. A conversa completa até o momento, composta por múltiplas mensagens anteriores, cada uma com um rótulo de posicionamento (CONCORDA, DISCORDA ou OUTROS), que indica como cada uma se posicionou em relação à sua própria mensagem de referência.
2. A mensagem de referência, que é o ponto de partida para a análise da mensagem atual.

Sua tarefa é classificar o posicionamento da mensagem atual em relação à mensagem de referência, com base no conteúdo e no contexto fornecido.

Rótulos possíveis:
1. CONCORDA – A mensagem atual expressa apoio, reforço ou alinhamento com o posicionamento da mensagem de referência. Pode repetir ideias, defender o mesmo ponto de vista ou responder positivamente à argumentação.
2. DISCORDA – A mensagem atual refuta, contesta ou critica o ponto de vista da mensagem de referência. Pode conter contra-argumentos, ironia crítica ou rejeição explícita das ideias anteriores.
3. OUTROS – A mensagem atual não expressa um posicionamento claro. Pode conter perguntas, pedidos de definição, comentários irrelevantes ou neutros, mudança de assunto ou ironia ambígua.

Instruções:
- Utilize a mensagem de referência como âncora para o julgamento.
- As mensagens anteriores (com seus respectivos rótulos) servem como contexto auxiliar, ajudando a interpretar o tom e a progressão da conversa.
- Não infira sentimentos nem intenções além do necessário: baseie-se em evidências explícitas ou logicamente implícitas.
- Quando houver ambiguidade ou neutralidade, classifique como OUTROS.

Formato de resposta:
```
CLASSIFICAÇÃO: [CONCORDA / DISCORDA / OUTROS]
JUSTIFICATIVA: [1 a 3 frases explicando de forma clara e objetiva o motivo da classificação]
```

Exemplos:
CONCORDA
Mensagem de referência: Eu acho que a criminalização do aborto é um exemplo claro de como a religião pode influenciar a política de forma injusta.
Mensagem atual: Apontar que um feto é uma vida não é suficiente para criminalizar o aborto. Precisamos de critérios objetivos.
Justificativa: A mensagem atual concorda com o argumento da referência ao defender a descriminalização com base em critérios objetivos, reforçando a crítica à influência religiosa.

DISCORDA
Mensagem de referência: O aborto não é aceitável pois um feto é uma vida.
Mensagem atual: O DNA não é critério suficiente para definir humanidade, e o direito à vida deveria considerar outras características.
Justificativa: A mensagem atual contradiz diretamente o argumento da referência, ao questionar a base para considerar o feto como portador de direitos.

OUTROS
Mensagem de referência: A relativização do direito à vida abre portas para outros crimes, como o estupro.
Mensagem atual: Defina "espécie Homo sapiens".
Justificativa: A mensagem atual não se posiciona sobre o conteúdo da referência, apenas solicita uma definição.
"""

#### Modelo do prompt user:
"""
### CONVERSA (com rótulos anteriores):\n\
[1] Pessoa A - RAIZ: Eu acho que a descriminalização do aborto é uma questão de saúde pública.\n\
[2] Pessoa B - RESPONDE A [1]: Mas isso ignora o direito do feto à vida. [DISCORDA]\n\
[3] Pessoa C - RESPONDE A [2]: Nem todo feto é viável; devemos considerar o caso da mãe. [CONCORDA]\n\
[4] Pessoa D - RESPONDE A [3]: Você poderia explicar o que você quer dizer com 'viável'? [OUTROS]\n\
\n\
### MENSAGEM ATUAL:\n\
Pessoa E - RESPONDE A [2]: Direito à vida de quem exatamente? Porque quando a mãe morre por um aborto clandestino, ninguém parece se importar.\n\
\n\
### MENSAGEM DE REFERÊNCIA:\n\
[1] Pessoa A - RAIZ: Eu acho que a descriminalização do aborto é uma questão de saúde pública.\n\
\n\
Classifique o posicionamento da **mensagem atual** com relação à **mensagem de referência**. Leve em consideração todo o contexto.
"""

# Criação do prompt com os dados
Aqui o foco é realizar um estudo da melhor maneira possível de utilizar os dados para chegar no prompt user pensado e estruturado

In [13]:
from collections import defaultdict

# Estrutura por conversa
conversas = defaultdict(list)

# Supondo que 'df' é seu DataFrame Pandas
for _, row in dados.iterrows():
    target_id = row["target_id"]
    mensagem = {
        "id": row["id"],
        "parent_id": row["parent_id"],
        "author": row["author"],
        "message": row["message"],
        "parent_message": row["parent_message"],
        "label": row["label"],
    }
    conversas[target_id].append(mensagem)

In [14]:
conversas

defaultdict(list,
            {1: [{'id': '2',
               'parent_id': '1',
               'author': 'Mary_Lane',
               'message': 'É engraçado que, se formos seguir a lógica do "ninguém deve ser obrigado a nada", um homem que engravidou alguma mulher sem querer, realmente não deveria ser obrigado a assumir a criança. Mas nessa sociedade que vivemos, mulher é café com leite e só quer direitos, sem responsabilidades ou sem dar os mesmos direitos ao homem.',
               'parent_message': 'E a paternidade, Sra. Ministra ? Considerando que vivemos em um país onde homens e mulheres são iguais perante a lei, homens não deveriam ser obrigados por lei a assumir um filho fruto de uma gravidez indesejada. Claro, isso em um contexto onde as mulheres podem abortar, e com isso se livrarem dos ônus de uma gravidez indesejada.',
               'label': 'Concorda'},
              {'id': '3',
               'parent_id': '2',
               'author': 'Karen_Poole',
               'messag

In [4]:
# A fazeres:
    # montar as conversas inteiras
        # creio que o target id possa ser útil para agrupar todas as mensagens de uma mesma conversa
    # descobrir o tamanho máximo de tokens e fazer uma função que verifica se o prompt está passando isso, se estiver passando, tratar de alguma maneira para não ter que ajustar manualmente
    # criar o prompt de system e o prompt com a mensagem
    # criar um método que extrai a resposta da saída (a saída precisa de tratamento/normalização)
    # criar uma função que armazena a resposta que o chatbot deu
    # realizar testes com tudo já automatizado (fazer para uns 20 rótulos, isso ja vai demorar bastante)

# testar o que a seguinte opção faz:
    # structured_outputs - JSON schema enforcement

# Feitos:
    # montar o prompt system
    # montar o prompt user
    